# Libs

In [2]:
!pip install pyspark==3.5.5 delta-spark==3.0.0

In [5]:
%pip install sentence-transformers
%pip install einops

Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 835.9 kB/s eta 0:00:000:00:01
Note: you may need to restart the kernel to use updated packages.


In [28]:
%pip install boto3
%pip install custom_st

Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement custom_st (from versions: none)
ERROR: No matching distribution found for custom_st
Note: you may need to restart the kernel to use updated packages.


# Spark

In [10]:
from pyspark.sql import DataFrame, SparkSession
from delta import DeltaTable
import pyspark.sql.functions as F
import pyspark.sql.types as T
import os
import socket
notebook_ip = socket.gethostbyname(socket.gethostname())
builder = SparkSession.builder.master('local[2]')
packages = [
    'io.delta:delta-spark_2.12:3.0.0',
    'org.apache.hadoop:hadoop-aws:3.3.4'
]

builder = (builder.config("spark.sql.parquet.int96RebaseModeInRead", "CORRECTED")
    .config("spark.sql.parquet.int96RebaseModeInWrite", "CORRECTED")
    .config("spark.sql.parquet.datetimeRebaseModeInRead", "CORRECTED")
    .config("spark.sql.parquet.datetimeRebaseModeInWrite", "CORRECTED")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.hadoop.fs.s3a.endpoint", "10.0.0.21:32222")
    .config("spark.hadoop.fs.s3a.access.key", "B53zPFuC9ScfhJV9")
    .config(
            "spark.hadoop.fs.s3a.secret.key", "h7jAiZty4JqAGBSEN0M1LqgHyWc18otP"
    )
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config(
        "spark.hadoop.fs.s3a.aws.credentials.provider",
        "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider",
    )
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
    .config(
        "spark.driver.maxResultSize",
        "16G",
    )
    .config(
        "spark.jars.packages",
        "io.delta:delta-spark_2.12:3.0.0,org.elasticsearch:elasticsearch-spark-30_2.12:7.13.1,org.apache.hadoop:hadoop-aws:3.2.4,org.postgresql:postgresql:42.6.0",
    )
    .config(
        "spark.jars.repositories",
        "https://repo1.maven.org/maven2",
    )
    .config(
        "spark.driver.extraJavaOptions",
        "-Xss4M",
    )
    .config(
        "spark.executor.extraJavaOptions",
        "-Xss4M",
    )
    .config(
        "spark.python.profile.memory",
        "true"
    )
    .config(
        "spark.sql.execution.arrow.pyspark.selfDestruct.enabled",
        "true"
    )
    .config(
        "spark.sql.execution.arrow.pyspark.enabled",
        "true"
    )
    .config(
        "spark.sql.execution.pythonUDF.arrow.enabled",
        "true"
    )
    .config(
        "spark.databricks.delta.retentionDurationCheck.enabled",
        "false"
    )
)
spark = builder.getOrCreate()

# Model

In [78]:
import os
import shutil
import zipfile
import boto3
from sentence_transformers import SentenceTransformer
from huggingface_hub import snapshot_download

class ModelLoader:
    def __init__(
        self,
        model_name: str,
        bucket: str,
        aws_access_key_id: str,
        aws_secret_access_key: str,
        endpoint_url: str = None,
        local_cache_base: str = None,
    ):
        self.model_name = model_name
        safe_model_name = model_name.replace("/", "-")
        self.local_cache_dir = os.path.expanduser(
            local_cache_base if local_cache_base else f"~/models/{safe_model_name}"
        )
        self.local_zip_path = self.local_cache_dir + ".zip"
        self.bucket = bucket
        self.s3_zip_key = f"models/{safe_model_name}.zip"

        boto3_params = {
            "service_name": "s3",
            "aws_access_key_id": aws_access_key_id,
            "aws_secret_access_key": aws_secret_access_key,
        }
        if endpoint_url:
            boto3_params["endpoint_url"] = endpoint_url

        self.s3_client = boto3.client(**boto3_params)

    def check_local_model(self):
        return os.path.exists(self.local_cache_dir) and len(os.listdir(self.local_cache_dir)) > 0

    def check_s3_zip_exists(self):
        try:
            self.s3_client.head_object(Bucket=self.bucket, Key=self.s3_zip_key)
            return True
        except self.s3_client.exceptions.ClientError:
            return False

    def zip_folder(self, folder_path, zip_path):
        print(f"Zipping folder {folder_path} -> {zip_path}")
        with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for root, _, files in os.walk(folder_path):
                for file in files:
                    full_path = os.path.join(root, file)
                    rel_path = os.path.relpath(full_path, folder_path)
                    zipf.write(full_path, rel_path)

    def unzip_file(self, zip_path, extract_to):
        print(f"Unzipping file {zip_path} -> {extract_to}")
        with zipfile.ZipFile(zip_path, 'r') as zipf:
            zipf.extractall(extract_to)

    def upload_zip_to_s3(self):
        print(f"Uploading {self.local_zip_path} to s3://{self.bucket}/{self.s3_zip_key}")
        self.s3_client.upload_file(self.local_zip_path, self.bucket, self.s3_zip_key)

    def download_zip_from_s3(self):
        print(f"Downloading s3://{self.bucket}/{self.s3_zip_key} to {self.local_zip_path}")
        os.makedirs(os.path.dirname(self.local_zip_path), exist_ok=True)
        self.s3_client.download_file(self.bucket, self.s3_zip_key, self.local_zip_path)

    def load(self):
        if self.check_local_model():
            print(f"Load model từ local cache: {self.local_cache_dir}")
            model = SentenceTransformer(self.local_cache_dir, trust_remote_code=True)
            return model

        print("Local cache chưa có, kiểm tra file zip trên S3...")
        if self.check_s3_zip_exists():
            print("Tải file zip model từ S3...")
            if os.path.exists(self.local_cache_dir):
                shutil.rmtree(self.local_cache_dir)
            self.download_zip_from_s3()
            self.unzip_file(self.local_zip_path, self.local_cache_dir)
            model = SentenceTransformer(self.local_cache_dir, trust_remote_code=True)
            return model

        print("Không có model trên local và S3, tải từ Huggingface Hub...")
        cache_dir = snapshot_download(self.model_name)
        print(f"Tải model cache về: {cache_dir}")

        if os.path.exists(self.local_cache_dir):
            shutil.rmtree(self.local_cache_dir)
        shutil.copytree(cache_dir, self.local_cache_dir)
        print(f"Copy cache model sang local cache folder: {self.local_cache_dir}")

        self.zip_folder(self.local_cache_dir, self.local_zip_path)
        self.upload_zip_to_s3()

        model = SentenceTransformer(self.local_cache_dir, trust_remote_code=True)
        return model




In [80]:
loader = ModelLoader(
    model_name="jinaai/jina-embeddings-v3",
    bucket="salessmart-warehouse",
    aws_access_key_id="B53zPFuC9ScfhJV9",
    aws_secret_access_key="h7jAiZty4JqAGBSEN0M1LqgHyWc18otP",
    endpoint_url="http://10.0.0.21:32222",  # Có thể bỏ nếu dùng AWS S3 chuẩn
)

model = loader.load()

# Dùng model embed câu ví dụ:
sentences = ["Hello world", "How are you?"]
embeddings = model.encode(sentences)
print(embeddings)


Load model từ local cache: /home/jovyan/models/jinaai-jina-embeddings-v3
[[ 0.09447937 -0.08225057  0.14750032 ...  0.01968503 -0.00741931
  -0.00151661]
 [ 0.0501683  -0.12939204  0.01436066 ...  0.01016663 -0.00945055
  -0.0182712 ]]


# Main

In [92]:
from delta.tables import DeltaTable
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, FloatType
import pandas as pd

master_companies_table = "s3a://salessmart-warehouse/table/master/company"
table = DeltaTable.forPath(spark, master_companies_table)
table_df = table.toDF()

df = (
    table_df
    .filter(table_df['business_content'].isNotNull())
    .limit(200)  # lấy sớm để giảm dữ liệu
    .select('corporate_number', 'name', 'business_content', 'meta_keywords', 'meta_descriptions', 'address')
    .repartition(8)  # tuỳ nhu cầu tài nguyên
)

schema = StructType([
    StructField('corporate_number', StringType()),
    StructField('embedding', ArrayType(FloatType()))
])

def embed_batch(pdf_iter):
    loader = ModelLoader(
    model_name="jinaai/jina-embeddings-v3",
    bucket="salessmart-warehouse",
    aws_access_key_id="B53zPFuC9ScfhJV9",
    aws_secret_access_key="h7jAiZty4JqAGBSEN0M1LqgHyWc18otP",
    endpoint_url="http://10.0.0.21:32222",  # Có thể bỏ nếu dùng AWS S3 chuẩn
    )

    model = loader.load()
    for pdf in pdf_iter:
        contents = (
            "企業名：" + pdf['name'].fillna('') + "\n" +
            "住所：" + pdf['address'].fillna('') + "\n" +
            "企業概要：" + pdf['business_content'].fillna('') + "\n" +
            "メタキーワード：" + pdf['meta_keywords'].fillna('') + "\n" +
            "メタディスクリプション：" + pdf['meta_descriptions'].fillna('')
        )
        embeddings = model.encode(contents.tolist(), task='retrieval.passage')
        yield pd.DataFrame({
            'corporate_number': pdf['corporate_number'],
            'embedding': [emb.tolist() for emb in embeddings]
        })

df_embeddings = df.mapInPandas(embed_batch, schema)

df_embeddings.write.mode("overwrite").parquet("s3a://salessmart-warehouse/test_company_embedding_7/")


/opt/conda/lib/python3.11/site-packages/pyspark/sql/udf.py:350: UserWarning: Profiling UDFs with iterators input/output is not supported.
  warnings.warn(
ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [85]:
print(spark.version)


3.5.0


In [ ]:
from delta.tables import DeltaTable
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, FloatType
import pandas as pd
import time

# Load DeltaTable
master_companies_table = "s3a://salessmart-warehouse/table/master/company"
table = DeltaTable.forPath(spark, master_companies_table)
table_df = table.toDF()

# Chọn cột, lọc null, giới hạn 200 dòng, repartition (tuỳ chỉnh)
df = (
    table_df
    .filter(table_df['business_content'].isNotNull())
    .limit(200)
    .select('corporate_number', 'name', 'business_content', 'meta_keywords', 'meta_descriptions', 'address')
    .repartition(8)
)

# Định nghĩa schema output cho mapInPandas
schema = StructType([
    StructField('corporate_number', StringType()),
    StructField('embedding', ArrayType(FloatType()))
])

def embed_batch(pdf_iter):
    loader = ModelLoader(
    model_name="jinaai/jina-embeddings-v3",
    bucket="salessmart-warehouse",
    aws_access_key_id="B53zPFuC9ScfhJV9",
    aws_secret_access_key="h7jAiZty4JqAGBSEN0M1LqgHyWc18otP",
    endpoint_url="http://10.0.0.21:32222",  # Có thể bỏ nếu dùng AWS S3 chuẩn
    )
    start_load = time.time()
    

    model = loader.load()
    print(f"[Executor] Model loaded in {time.time() - start_load:.2f} seconds")

    for pdf in pdf_iter:
        batch_size = len(pdf)
        print(f"[Executor] Processing batch of size {batch_size}")

        start_encode = time.time()
        contents = (
            "企業名：" + pdf['name'].fillna('') + "\n" +
            "住所：" + pdf['address'].fillna('') + "\n" +
            "企業概要：" + pdf['business_content'].fillna('') + "\n" +
            "メタキーワード：" + pdf['meta_keywords'].fillna('') + "\n" +
            "メタディスクリプション：" + pdf['meta_descriptions'].fillna('')
        )
        embeddings = model.encode(contents.tolist(), task='retrieval.passage')
        print(f"[Executor] Batch embedding done in {time.time() - start_encode:.2f} seconds")

        yield pd.DataFrame({
            'corporate_number': pdf['corporate_number'],
            'embedding': [emb.tolist() for emb in embeddings]
        })

# Chạy mapInPandas
df_embeddings = df.mapInPandas(embed_batch, schema)

# Ghi kết quả parquet ra S3
df_embeddings.write.mode("overwrite").parquet("s3a://salessmart-warehouse/test_company_embedding_7/")


/opt/conda/lib/python3.11/site-packages/pyspark/sql/udf.py:350: UserWarning: Profiling UDFs with iterators input/output is not supported.
  warnings.warn(


In [ ]:
from delta.tables import DeltaTable
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, FloatType
import pandas as pd
import time

# Đường dẫn DeltaTable trên S3
master_companies_table = "s3a://salessmart-warehouse/table/master/company"
table = DeltaTable.forPath(spark, master_companies_table)
table_df = table.toDF()

# Chọn các cột theo đúng tên, lọc null business_content, limit 200 dòng, repartition phù hợp
df = (
    table_df
    .filter(table_df['business_content'].isNotNull())
    .limit(200)
    .select(
        'corporate_number', 'name', 'business_content', 'meta_keywords', 'meta_descriptions',
        'address', 'industry_code', 'capital', 'revenue', 'employee_count', 'listing_market_code'
    )
    .repartition(8)
)

# Định nghĩa schema output của mapInPandas (corporate_number và embedding vector)
schema = StructType([
    StructField('corporate_number', StringType()),
    StructField('embedding', ArrayType(FloatType()))
])

def embed_batch(pdf_iter):
    # Import loader module của bạn để load model trong executor
    from your_loader_module import loader  # Thay bằng module thực tế

    start_load = time.time()
    model = loader.load()
    print(f"[Executor] Model loaded in {time.time() - start_load:.2f} seconds", flush=True)

    for pdf in pdf_iter:
        batch_size = len(pdf)
        print(f"[Executor] Processing batch of size {batch_size}", flush=True)

        start_encode = time.time()
        contents = (
            "企業名：" + pdf['name'].fillna('') + "\n" +
            "住所：" + pdf['address'].fillna('') + "\n" +
            "企業概要：" + pdf['business_content'].fillna('') + "\n" +
            "メタキーワード：" + pdf['meta_keywords'].fillna('') + "\n" +
            "メタディスクリプション：" + pdf['meta_descriptions'].fillna('') + "\n" +
            "職種：" + pdf['industry_code'].fillna('') + "\n" +
            "資本金：" + pdf['capital'].fillna('') + "\n" +
            "売上高：" + pdf['revenue'].fillna('') + "\n" +
            "従業員数：" + pdf['employee_count'].fillna('') + "\n" +
            "上場区分：" + pdf['listing_market_code'].fillna('')
        )
        embeddings = model.encode(contents.tolist(), task='retrieval.passage')
        print(f"[Executor] Batch embedding done in {time.time() - start_encode:.2f} seconds", flush=True)

        yield pd.DataFrame({
            'corporate_number': pdf['corporate_number'],
            'embedding': [emb.tolist() for emb in embeddings]
        })

# Áp dụng mapInPandas để chạy song song
df_embeddings = df.mapInPandas(embed_batch, schema)

# Ghi kết quả parquet ra S3
df_embeddings.write.mode("overwrite").parquet("s3a://salessmart-warehouse/test_company_embedding_7/")
